# Battery Analyzer - Basic Usage Example

이 노트북은 Battery Analyzer의 기본 사용법을 보여줍니다.

## 목차
1. 설정 생성 (Configuration)
2. 데이터 로딩 (Data Loading)
3. Cycle 분석 (Cycle Analysis)
4. Profile 분석 (Profile Analysis)
5. 결과 내보내기 (Export)

## 1. 설정 생성 (Configuration)

분석에 필요한 설정을 생성합니다.

In [ ]:
import sys
sys.path.append('..')

from battery_analyzer.config import (
    AnalysisConfig,
    PathConfig,
    CapacityConfig,
    CycleConfig,
    ProfileConfig,
    ExportConfig,
    CapacityMode,
    DCIRMode,
    ProfileLayout,
)

# 경로 설정
path_config = PathConfig(
    use_tsv=False,
    folder_paths=[
        r"C:\Users\Ryu\Desktop\battery_test_data\sample1",
    ]
)

# 용량 설정 (자동 감지 모드)
capacity_config = CapacityConfig(
    mode=CapacityMode.AUTO_CRATE,
    c_rate=0.2,
)

# Cycle 분석 설정
cycle_config = CycleConfig(
    cycle_numbers=[2, 3, 4, 5],  # 개별 사이클
    x_max=0,  # 자동
    y_max=1.10,
    y_min=0.65,
    dcir_mode=DCIRMode.RSS,
)

# Profile 분석 설정
profile_config = ProfileConfig(
    cycle_numbers=[2],
    layout=ProfileLayout.BY_CYCLE,
    voltage_min=2.5,
    voltage_max=4.7,
    smoothing=0,  # 자동
)

# 내보내기 설정
export_config = ExportConfig(
    save_excel=True,
    save_figure=True,
    excel_filename="cycle_analysis.xlsx",
    figure_filename="cycle_plot.png",
)

# 전체 설정 통합
config = AnalysisConfig(
    path=path_config,
    capacity=capacity_config,
    cycle=cycle_config,
    profile=profile_config,
    export=export_config,
)

# 설정 검증
config.validate()
print("✅ Configuration created and validated successfully")

## 2. 데이터 로딩 (Data Loading)

CycleDataContainer를 사용하여 전체 사이클을 1회 로드합니다.

In [ ]:
from battery_analyzer.data import (
    CycleDataContainer,
    DataLoaderFactory,
)
from battery_analyzer.utils import get_all_cycles

# 데이터 경로
data_path = config.path.folder_paths[0]

# 데이터 로더 생성 (자동 감지)
loader = DataLoaderFactory.create_loader(data_path)
print(f"📂 Detected cycler type: {loader.cycler_type.value.upper()}")

# 전체 사이클 범위 확인
try:
    cycle_range = get_all_cycles(data_path)
    print(f"📊 Available cycle range: {cycle_range[0]} to {cycle_range[1]}")
except Exception as e:
    print(f"⚠️ Could not detect cycle range: {e}")
    cycle_range = (1, 10)  # 기본값

# CycleDataContainer 생성 및 로드
container = CycleDataContainer()
print(f"\n⏳ Loading cycles {cycle_range[0]} to {cycle_range[1]}...")

try:
    container.load_all_cycles(data_path, loader, cycle_range)
    print(f"✅ Loaded {len(container)} cycles successfully")
    print(f"📋 Loaded cycle numbers: {container.get_loaded_cycle_numbers()}")
except Exception as e:
    print(f"❌ Error loading cycles: {e}")

## 3. 개별 사이클 조회 (Cycle Query)

로드된 사이클 데이터를 O(1) 시간에 조회합니다.

In [ ]:
# 단일 사이클 조회
if container.has_cycle(2):
    df_cycle2 = container.get_cycle(2)
    print(f"📈 Cycle 2 data shape: {df_cycle2.shape}")
    print(f"\nFirst 5 rows:")
    print(df_cycle2.head())

# 여러 사이클 조회
if all(container.has_cycle(c) for c in [2, 3, 4]):
    cycles_dict = container.get_cycles([2, 3, 4])
    print(f"\n✅ Retrieved {len(cycles_dict)} cycles")
    for cycle_no, df in cycles_dict.items():
        print(f"  - Cycle {cycle_no}: {len(df)} rows")

# 연속 범위 조회 (병합)
try:
    df_range = container.get_cycle_range(2, 4)
    print(f"\n📊 Merged cycle range 2-4: {df_range.shape}")
    print(f"Time range: {df_range['TimeMin'].min():.2f} to {df_range['TimeMin'].max():.2f} min")
except Exception as e:
    print(f"⚠️ Error retrieving cycle range: {e}")

## 4. 기본 통계 및 시각화

로드된 데이터의 기본 통계를 확인하고 간단한 시각화를 수행합니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Cycle 2 전압-시간 플롯
if container.has_cycle(2):
    df = container.get_cycle(2)
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # 전압-시간
    axes[0].plot(df['TimeMin'], df['Vol'], 'b-', linewidth=1.5)
    axes[0].set_xlabel('Time (min)')
    axes[0].set_ylabel('Voltage (V)')
    axes[0].set_title('Cycle 2: Voltage vs Time')
    axes[0].grid(True, alpha=0.3)
    
    # 전류-시간
    axes[1].plot(df['TimeMin'], df['Crate'], 'r-', linewidth=1.5)
    axes[1].set_xlabel('Time (min)')
    axes[1].set_ylabel('Current (C-rate)')
    axes[1].set_title('Cycle 2: Current vs Time')
    axes[1].grid(True, alpha=0.3)
    axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 기본 통계
    print("\n📊 Cycle 2 Statistics:")
    print(f"  - Duration: {df['TimeMin'].max():.2f} min")
    print(f"  - Voltage range: {df['Vol'].min():.3f} to {df['Vol'].max():.3f} V")
    print(f"  - Current range: {df['Crate'].min():.3f} to {df['Crate'].max():.3f} C-rate")

## 5. 용량 계산 예제

방전 용량 및 충전 용량을 계산합니다.

In [ ]:
def calculate_capacity(df, capacity_mah=58.0):
    """방전/충전 용량 계산"""
    # 시간 차이 계산
    df = df.copy()
    df['TimeDiff'] = df['TimeMin'].diff().fillna(0)
    
    # 방전 용량 (Crate < 0)
    discharge = df[df['Crate'] < 0].copy()
    discharge_capacity = abs(discharge['Crate'] * discharge['TimeDiff']).sum() * capacity_mah
    
    # 충전 용량 (Crate > 0)
    charge = df[df['Crate'] > 0].copy()
    charge_capacity = (charge['Crate'] * charge['TimeDiff']).sum() * capacity_mah
    
    # 효율
    efficiency = (discharge_capacity / charge_capacity * 100) if charge_capacity > 0 else 0
    
    return {
        'discharge_capacity': discharge_capacity,
        'charge_capacity': charge_capacity,
        'efficiency': efficiency,
    }

# 여러 사이클 용량 계산
if config.cycle and config.cycle.cycle_numbers:
    print("\n⚡ Capacity Analysis:")
    print(f"{'Cycle':<10} {'Discharge (mAh)':<20} {'Charge (mAh)':<20} {'Efficiency (%)':<15}")
    print("-" * 65)
    
    for cycle_no in config.cycle.cycle_numbers[:5]:  # 처음 5개만
        if container.has_cycle(cycle_no):
            df = container.get_cycle(cycle_no)
            result = calculate_capacity(df)
            print(f"{cycle_no:<10} {result['discharge_capacity']:<20.2f} {result['charge_capacity']:<20.2f} {result['efficiency']:<15.2f}")

## 6. 메타데이터 확인

컨테이너의 메타데이터를 확인합니다.

In [ ]:
# 메타데이터 출력
metadata = container.get_metadata()
print("\n📋 Container Metadata:")
for key, value in metadata.items():
    print(f"  - {key}: {value}")

# 컨테이너 정보
print(f"\n{container}")

## 7. 메모리 정리

작업 완료 후 컨테이너를 정리합니다.

In [ ]:
# 컨테이너 초기화
container.clear()
print(f"✅ Container cleared: {len(container)} cycles remaining")

## 정리

이 노트북에서는 다음 내용을 다뤘습니다:

1. ✅ 설정 생성 (PathConfig, CapacityConfig, CycleConfig, ProfileConfig)
2. ✅ 데이터 로더 자동 감지 (PNE/Toyo)
3. ✅ 전체 사이클 로드 (CycleDataContainer)
4. ✅ O(1) 사이클 조회 (get_cycle, get_cycles, get_cycle_range)
5. ✅ 기본 시각화 (전압/전류 플롯)
6. ✅ 용량 계산 (방전/충전/효율)

다음 단계:
- Phase 3.4: Cycle 분석 엔진 (Individual, Linked, Reliability)
- Phase 3.5: Profile 분석 엔진 (Step, Rate, Charge, Discharge, DCIR)
- Phase 3.6-3.7: Visualization 및 Export